In [4]:
from flask import Flask, request, jsonify
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler

# Initialize Flask app
app = Flask(__name__)

# Load the model
model = tf.keras.models.load_model('lstm_model.keras')

# Load and prepare the data
df = pd.read_csv('xrp_features.csv', index_col=0, parse_dates=True)
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(df['close'].values.reshape(-1, 1))

# Function to create dataset
def create_dataset(data, time_step=1):
    X = []
    for i in range(len(data) - time_step - 1):
        X.append(data[i:(i + time_step), 0])
    return np.array(X)

# Create dataset for prediction
time_step = 10
dataX = create_dataset(scaled_data, time_step)

@app.route('/predict', methods=['POST'])
def predict():
    # Get input data
    input_data = request.json['data']
    
    # Prepare input data for the model
    input_data = np.array(input_data).reshape(1, time_step, 1)
    input_data = scaler.transform(input_data.reshape(-1, 1)).reshape(1, time_step, 1)
    
    # Make prediction
    prediction = model.predict(input_data)
    prediction = scaler.inverse_transform(prediction)
    
    # Return prediction
    return jsonify({'prediction': prediction[0][0]})

if __name__ == '__main__':
    app.run()

 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
 * Restarting with watchdog (fsevents)
Traceback (most recent call last):
  File "/Users/seanfarquharson/anaconda3/lib/python3.11/site-packages/ipykernel_launcher.py", line 15, in <module>
    from ipykernel import kernelapp as app
  File "/Users/seanfarquharson/anaconda3/lib/python3.11/site-packages/ipykernel/__init__.py", line 5, in <module>
    from .connect import *  # noqa
    ^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/seanfarquharson/anaconda3/lib/python3.11/site-packages/ipykernel/connect.py", line 11, in <module>
    import jupyter_client
  File "/Users/seanfarquharson/anaconda3/lib/python3.11/site-packages/jupyter_client/__init__.py", line 4, in <module>
    from .connect import *
  File "/Users/seanfarquharson/anaconda3/lib/python3.11/site-packages/jupyter_client/connect.py", line 28, in <module>
    from jupyter_core.paths import jupyter_data_dir, jupyter_runtime_dir, secure_write
  File "/Users/seanfarquharson/anaconda3/li

SystemExit: 1

/Users/seanfarquharson/anaconda3/lib/python3.11/site-packages/IPython/core/interactiveshell.py:3513: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
